In [1]:
%load_ext autoreload
%autoreload 2

# Testing Notebook

In [2]:
import numpy as np
import h5py

In [3]:
wfndir = './WFN.h5'

In [4]:
wfn_ = h5py.File(wfndir, 'r')
print(wfn_.keys())

<KeysViewHDF5 ['mf_header', 'wfns']>


In [5]:
wfns_ = wfn_["wfns"]
mf_header = wfn_["mf_header"]
kpoints = mf_header["kpoints"]
gspace = mf_header["gspace"]
symmetry = mf_header["symmetry"]
crystal_ = mf_header["crystal"]

In [6]:
print(wfns_["coeffs"].shape) # Shape does not match the 'WFN.h5' docs. Shape is transposed

(30, 1, 35749, 2)


# Crystal

In [7]:
alat = np.array(crystal_["alat"])
nat = np.array(crystal_["nat"])
avec = np.array(crystal_["avec"])
atyp = np.array(crystal_["atyp"])
apos = np.array(crystal_["apos"])
print(alat)
print(nat)
print(avec)
print(atyp)
print(apos)

10.2
2
[[-0.5  0.   0.5]
 [ 0.   0.5  0.5]
 [-0.5  0.5  0. ]]
[14 14]
[[ 0.125 -0.125 -0.125]
 [-0.125  0.125  0.125]]


In [8]:
from quantum_masala.core import RealLattice, AtomBasis, Crystal

# Generating 'reallat'
reallat = RealLattice.from_alat(alat, *avec)

# Parsing 'atyp' and 'apos' to generate 'l_species'
isort = np.argsort(atyp)
l_typ, l_counts = np.unique(atyp, return_counts=True)
l_coords = np.split(apos[isort], np.cumsum(l_counts))

l_species = []
for ityp, typ_num in enumerate(l_typ):
    l_species.append(AtomBasis.from_alat(reallat, typ_num, None, None, *l_coords[ityp]))
    
# Finally creating 'Crystal' instance
crystal = Crystal(reallat, l_species)

In [9]:
reallat, recilat, l_species = crystal.reallat, crystal.recilat, crystal.l_species

print("Bravais Lattice")
print(f"Lattice parameter 'a': alat = {reallat.alat} bohr")
print("Axes of bravais lattice in units of alat:")
for i, ax in enumerate(reallat.axes_alat):
    print(f"a{i+1} = {ax}")

print("Checking additional values in bgw's 'wfn.h5'")
print("alat: ", np.allclose(reallat.alat, np.array(crystal_["alat"])))
print("avec: ", np.allclose(reallat.axes_alat, np.array(crystal_["avec"])))
print("celvol: ", np.allclose(reallat.cellvol, np.array(crystal_["celvol"])))
print("adot: ", np.allclose(reallat.adot, np.array(crystal_["adot"])))
print("-"*40)
print()

print("Reciprocal Lattice")
print(f"Lattice parameter 'b': blat = {recilat.tpiba} bohr^-1")
print("Axes of reciprocal lattice in units of tpiba:")
for i, bx in enumerate(recilat.axes_tpiba):
    print(f"b{i+1} = {bx}")

print("Checking additional values in bgw's 'wfn.h5'")
print("blat: ", np.allclose(recilat.tpiba, np.array(crystal_["blat"])))
print("bvec: ", np.allclose(recilat.axes_tpiba, np.array(crystal_["bvec"])))
print("recvol: ", np.allclose(recilat.cellvol, np.array(crystal_["recvol"])))
print("bdot: ", np.allclose(recilat.bdot, np.array(crystal_["bdot"])))
print("-"*40)
print()

print("Atoms in Unit Cell")
for isp, sp in enumerate(l_species):
    print(f"Species #{isp+1}: Atomic Number - {sp.label}")
    print(f"Coordinates in units of alat")
    for iat, r_alat in enumerate(sp.alat.T): # Note the transpose
        print(r_alat)
print("-"*40)
print()

Bravais Lattice
Lattice parameter 'a': alat = 10.2 bohr
Axes of bravais lattice in units of alat:
a1 = [-0.5  0.   0.5]
a2 = [0.  0.5 0.5]
a3 = [-0.5  0.5  0. ]
Checking additional values in bgw's 'wfn.h5'
alat:  True
avec:  True
celvol:  True
adot:  True
----------------------------------------

Reciprocal Lattice
Lattice parameter 'b': blat = 0.6159985595274104 bohr^-1
Axes of reciprocal lattice in units of tpiba:
b1 = [-1. -1.  1.]
b2 = [1. 1. 1.]
b3 = [-1.  1. -1.]
Checking additional values in bgw's 'wfn.h5'
blat:  True
bvec:  True
recvol:  True
bdot:  True
----------------------------------------

Atoms in Unit Cell
Species #1: Atomic Number - 14
Coordinates in units of alat
[ 0.125 -0.125 -0.125]
[-0.125  0.125  0.125]
----------------------------------------



# GSpace

In [10]:
ecutrho = np.array(gspace["ecutrho"])
FFTgrid = np.array(gspace["FFTgrid"])

In [11]:
from quantum_masala.core import GSpace
from quantum_masala.core.constants import E_ryd

grho = GSpace(crystal.recilat, ecutrho * E_ryd, FFTgrid.tolist())  
# NOTE: Check if FFTgrid needs to be reversed by generating WFN.h5 
# for a unequal mesh via QE input 'nr1, nr2, nr3' arguments

In [12]:
print(f"Number of vectors in G-Sphere: {grho.numg}")
print("Matches with 'WFN.h5': ", grho.numg == np.array(gspace["ng"]))
print()

components = np.array(gspace["components"])
isort_bgw = np.lexsort([components[:, 2], components[:, 1], components[:, 0]])
isort_py = np.lexsort([grho.cryst[2], grho.cryst[1], grho.cryst[0]])

print("Components match: ", np.allclose(components[isort_bgw], grho.cryst.T[isort_py]))

Number of vectors in G-Sphere: 4477
Matches with 'WFN.h5':  True

Components match:  True


In [13]:
# Generating a mapping between 'grho.cryst' and 'gspace["components"]'
idx_grho_bgw2pw = np.empty(grho.numg, dtype='i4')
idx_grho_bgw2pw[isort_py] = np.arange(grho.numg, dtype='i4')[isort_bgw]

print("Testing generated mapping from bgw to pw")
print("Match: ", np.allclose(grho.cryst.T, components[idx_grho_bgw2pw]))

Testing generated mapping from bgw to pw
Match:  True


# KPoints

In [14]:
w = np.array(kpoints["w"])
rk = np.array(kpoints["rk"]) # determined to be in cryst

In [15]:
from quantum_masala.core import KPoints

kpts = KPoints.from_cryst(reallat, *zip(rk, w))

In [16]:
print("Checking KPoints instance")
print("Number: ", np.allclose(kpts.numk, np.array(kpoints["nrk"])))
print("Coordinates: ", np.allclose(kpts.cryst.T, np.array(kpoints["rk"])))
print("Weights: ", np.allclose(kpts.weights, np.array(kpoints["w"])))

Checking KPoints instance
Number:  True
Coordinates:  True
Weights:  True


# GSpaceWfn

In [17]:
ecutwfc = np.array(kpoints["ecutwfc"])
print(ecutwfc)

25.0


In [18]:
from quantum_masala.core import GSpaceWfn

wfn_gspc = GSpace(recilat, 4 * (ecutwfc * E_ryd), grho.grid_shape)
l_gwfn = [GSpaceWfn(wfn_gspc, k_cryst) for k_cryst in kpts.cryst.T]

In [19]:
print("ngk: ", np.allclose([gwfn.numgk for gwfn in l_gwfn], np.array(kpoints["ngk"])))

ngk:  True


In [20]:
gvecs = np.array(wfns_["gvecs"])
l_gk_bgw = np.split(np.array(wfns_["gvecs"]), np.cumsum(np.array(kpoints["ngk"])[:-1]))

for i, gwfn in enumerate(l_gwfn):
    print(f"Validating 'gwfn' for k-point #{i+1}")
    gcryst_py = gwfn.g_cryst
    gcryst_bgw = l_gk_bgw[i]
    isort_bgw = np.lexsort([gcryst_bgw[:, 2], gcryst_bgw[:, 1], gcryst_bgw[:, 0]])
    isort_py = np.lexsort([gcryst_py[2], gcryst_py[1], gcryst_py[0]])
    print("Components match: ", np.allclose(gcryst_bgw[isort_bgw], gcryst_py.T[isort_py]))
    idxbgw2pw = np.empty(gwfn.numgk, dtype='i4')
    idxbgw2pw[isort_py] = np.arange(gwfn.numgk, dtype='i4')[isort_bgw]
    gwfn.idxbgw2pw = idxbgw2pw
    print("Generated mapping is correct: ", np.allclose(gwfn.g_cryst.T, gcryst_bgw[gwfn.idxbgw2pw]))    
    print()


Validating 'gwfn' for k-point #1
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #2
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #3
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #4
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #5
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #6
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #7
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #8
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #9
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #10
Components match:  True
Generated mapping is correct:  True

Validating 'gwfn' for k-point #11
Components match:  True
G

# Loading Wavefunctions

In [21]:
nspin = np.array(kpoints["nspin"])
nspinor = np.array(kpoints["nspinor"]) # Raise error when it is not 1
mnband = np.array(kpoints["mnband"])
el = np.array(kpoints["el"])
occ = np.array(kpoints["occ"])

In [22]:
from quantum_masala.core import WavefunK

WfnK = WavefunK(wfn_gspc, int(nspin), int(mnband))
l_wfn = [WfnK(k_cryst, k_weight, idxk) for idxk, (k_cryst, k_weight) in enumerate(kpts)]

gvecs = np.array(wfn_["wfns/gvecs"])
l_gk_bgw = np.split(np.array(wfn_["wfns/gvecs"]), np.cumsum(np.array(kpoints["ngk"])[:-1]))
print(wfns_["coeffs"].shape)
l_coeffs = np.split(
    np.array(wfns_["coeffs"]).view('c16').reshape((mnband, nspin, -1)).transpose((1,0,2)),
    np.cumsum(np.array(kpoints["ngk"][:-1])),
    axis=2
    )

for ik, wfn in enumerate(l_wfn):
    wfn.evl[:] = kpoints["el"][:, ik, :]
    wfn.occ[:] = kpoints["occ"][:, ik, :]
    
    # Duplicated code for reference
    gwfn = wfn.gwfc  # Yet to refactor the RHS name from gwfc -> gwfn
    gcryst_py = gwfn.g_cryst
    gcryst_bgw = l_gk_bgw[ik]
    isort_bgw = np.lexsort([gcryst_bgw[:, 2], gcryst_bgw[:, 1], gcryst_bgw[:, 0]])
    isort_py = np.lexsort([gcryst_py[2], gcryst_py[1], gcryst_py[0]])
    print("Components match: ", np.allclose(gcryst_bgw[isort_bgw], gcryst_py.T[isort_py]))
    idxbgw2pw = np.empty(gwfn.numgk, dtype='i4')
    idxbgw2pw[isort_py] = np.arange(gwfn.numgk, dtype='i4')[isort_bgw]
    gwfn.idxbgw2pw = idxbgw2pw
    print("Generated mapping is correct: ", np.allclose(gwfn.g_cryst.T, gcryst_bgw[gwfn.idxbgw2pw]))
    wfn.evc_gk[:] = l_coeffs[ik][:, :, gwfn.idxbgw2pw]
    print()
print("Loaded WFC Data")

(30, 1, 35749, 2)
Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Components match:  True
Generated mapping is correct:  True

Compon